# Atividade II

Crie uma tabela com os dados de Retorno, Volatilidade, Alfa, Beta, Sharpe e Sortino para cada um dos portfólios.

In [ ]:
import pandas as pd
import numpy as np

import config as cfg
import helpers as hlp

In [ ]:
# Lendo a lisrta de portfólios
df_portfolio_list = pd.read_csv(f'{cfg.PATH_RESULTADOS}/portfolio_files.csv')

df_portfolio_list.head()

## 1. Calculando as variáveis para o portfólio Blue Chips

In [ ]:
# Recuperando o arquivo
portfolio_name = df_portfolio_list['Portfolio'].iloc[0]
file_name = df_portfolio_list['File Name'].iloc[0]

df_blue_chips = pd.read_parquet(f"{cfg.PATH_CALCULATED_PORTFOLIO_DIR}/{file_name}", columns=['TOTAL AMOUNT']).rename(columns={'TOTAL AMOUNT': portfolio_name})

df_blue_chips.head()

In [ ]:
def log_return(df, column):
    return np.log(df[column]/df[column].shift(1))

In [ ]:
LOG_COLUMN_NAME = "Log Return"

df_blue_chips[LOG_COLUMN_NAME] = log_return(df_blue_chips, portfolio_name)

In [ ]:
# Retorno acumulado do fundo
blue_chips_return = df_blue_chips[LOG_COLUMN_NAME].sum()

# A volatilidade 
blue_chips_vol = df_blue_chips[LOG_COLUMN_NAME].std()

In [ ]:
print(f'Log Return Acum => {blue_chips_return * 100}')
print(f'Volatility => {blue_chips_vol * 100}')

In [ ]:
# Buscando os dados da IBOV e do DI do período
IBOV = '.BVSP'
DI_INDEX = 'DI_INDEX'

df_prelast = pd.read_parquet(cfg.PATH_PRELAST_PARQUET, columns=['Data', IBOV, DI_INDEX])
df_prelast.set_index('Data', inplace=True)

In [ ]:
# Calulando os retornos
IBOV_RETURN = f'{IBOV} Log Return'
DI_INDEX_RETURN = f'{DI_INDEX} Log Return'

df_prelast[IBOV_RETURN] = log_return(df_prelast, IBOV)
df_prelast[DI_INDEX_RETURN] = log_return(df_prelast, DI_INDEX)

bvsp_return = df_prelast[IBOV_RETURN].sum()
di_return = df_prelast[DI_INDEX_RETURN].sum()

print(f'BVSP Log Return Acum => {bvsp_return * 100}')
print(f'DI_INDEX Log Return Acum => {di_return * 100}')

In [ ]:
# Gerando data frame para cálculo do beta
base_ativos = pd.merge(df_prelast[IBOV_RETURN], df_blue_chips[LOG_COLUMN_NAME], left_index=True, right_index=True)
base_ativos.columns = [IBOV, portfolio_name]
base_ativos.dropna(inplace=True)

base_ativos.head()

In [ ]:
# Calcule a matriz de covariância para o mercado e o ativo
#cov_matrix = np.cov(base_ativos.iloc[:,0], base_ativos.iloc[:,1])
cov_matrix = np.cov(base_ativos[IBOV], base_ativos[portfolio_name])

# Extraia a covariância entre o mercado e o ativo
market_asset_cov = cov_matrix[0,1]

# Calcule o beta do ativo (ou seja, a sensibilidade dos retornos do ativo aos retornos do mercado)
beta = market_asset_cov / np.var(base_ativos[IBOV])

In [ ]:
expected_return = di_return + beta * (bvsp_return - di_return)

alfa = blue_chips_return - expected_return

In [ ]:
# Calculando indice sharp
sharpe_index = (blue_chips_return - di_return)/blue_chips_vol

# Sortino
avg_return = df_blue_chips[LOG_COLUMN_NAME].mean()
downside_deviation = np.sqrt(np.mean(np.square(np.minimum(df_blue_chips[LOG_COLUMN_NAME] - avg_return, 0))))
sortino_index = (blue_chips_return - di_return)/downside_deviation

In [ ]:
print(f'Log Return Acum => {blue_chips_return * 100}')
print(f'Volatility => {blue_chips_vol * 100}')
print(f'expected return => {expected_return * 100}')
print(f'alfa => {alfa * 100}')
print(f'beta => {beta}')
print(f'sharpe index => {sharpe_index}')
print(f'sortino index => {sortino_index}')

## 2. Calculando para todos os fundos e gerando a tabela sumarizada

In [ ]:
def calculate_log_return(df, column):
    return np.log(df[column]/df[column].shift(1))

def calculate_beta(df, market_name, asset_name):
    # Calcule a matriz de covariância para o mercado e o ativo
    cov_matrix = np.cov(df[market_name], df[asset_name])

    # Extraia a covariância entre o mercado e o ativo
    market_asset_cov = cov_matrix[0,1]

    # Calcule o beta do ativo (ou seja, a sensibilidade dos retornos do ativo aos retornos do mercado)
    return market_asset_cov / np.var(df[market_name])

def calculate_expected_return(free_risk_return, beta, benchmark_return):
    return free_risk_return + beta * (benchmark_return - free_risk_return)

def calculate_alfa(asset_return, expected_return):
    return asset_return - expected_return

def calculate_downside_deviation(df, asset_name):
    avg_return = df[asset_name].mean()
    return np.sqrt(np.mean(np.square(np.minimum(df[asset_name] - avg_return, 0))))

# As formulas dos índices Sharp e Sortino são iguais só mudando o desvio (um usa a vol o outro desvio negativo)
def caculate_sharpe_sortino_index(asset_return, free_risk_return, deviation):
    return (asset_return - free_risk_return) / deviation


In [ ]:
# ------------------------------------------------------------------------------------------------
# Gerando inicializando um data fram dos retornos com os retornos do IBOV e do DI
# ------------------------------------------------------------------------------------------------
IBOV = '.BVSP'
DI_INDEX = 'DI_INDEX'

# Buscando os dados dos dois índices
df_prelast = pd.read_parquet(cfg.PATH_PRELAST_PARQUET, columns=['Data', IBOV, DI_INDEX])
df_prelast.set_index('Data', inplace=True)

df_returns = pd.DataFrame()
df_returns[IBOV] = calculate_log_return(df_prelast, IBOV)
df_returns[DI_INDEX] = calculate_log_return(df_prelast, DI_INDEX)

df_returns.head()

In [ ]:
#Acrescentando o retorno dos portfólios
for index, row in df_portfolio_list.iterrows():
    portfolio_name = row['Portfolio']
    file_name = row['File Name']

    df = pd.read_parquet(f"{cfg.PATH_CALCULATED_PORTFOLIO_DIR}/{file_name}", columns=['TOTAL AMOUNT']).rename(columns={'TOTAL AMOUNT': portfolio_name})
    
    df_returns[portfolio_name] = calculate_log_return(df, portfolio_name)

df_returns.head()

In [ ]:
# Eliminando o N/A
df_returns.dropna(inplace = True)

In [ ]:
bvsp_return = df_returns[IBOV].sum()
di_return = df_returns[DI_INDEX].sum()

portfolio_indexes_list = [{
            'Portfolio' : IBOV, 
            'Retorno' : bvsp_return * 100
            },
            {
            'Portfolio' : DI_INDEX, 
            'Retorno' : di_return * 100
            }
            ]

for index, row in df_portfolio_list.iterrows():
    portfolio_name = row['Portfolio']

    # Calculando indicadores do Portfolio
    acum_return = df_returns[portfolio_name].sum()
    vol_portfolio = df_returns[portfolio_name].std()
    downside_deviation = calculate_downside_deviation(df_returns, portfolio_name)
    beta = calculate_beta(df_returns, IBOV, portfolio_name)
    expected_return = calculate_expected_return(di_return, beta, bvsp_return)
    alfa = calculate_alfa(acum_return,expected_return)
    sharpe_index = caculate_sharpe_sortino_index(acum_return, di_return, vol_portfolio)
    sortino_index = caculate_sharpe_sortino_index(acum_return, di_return, downside_deviation)

    row = {
            'Portfolio' : portfolio_name, 
            'Retorno' : acum_return * 100,
            'Volatilidade' : vol_portfolio * 100, 
            'Retorno Experado' : expected_return * 100,
            'Alfa' : alfa * 100,
            'Beta' : beta,
            'Sharpe' : sharpe_index,
            'Sortino' : sortino_index
        }

    portfolio_indexes_list.append(row)

df_portfolio_index = pd.DataFrame(portfolio_indexes_list)
df_portfolio_index.head()

In [ ]:
# Salvando os dados
hlp.save_to_parquet(df_returns, cfg.PATH_TABELA_RETORNOS)
hlp.save_to_parquet(df_portfolio_index, cfg.PATH_TABELA_INDICADORES_PORTFOLIO)